In [17]:
!pip install simple_salesforce

In [1]:
import json
from sqlalchemy import create_engine
from datetime import date
import pandas as pd
import pymysql
pymysql.install_as_MySQLdb()

In [2]:
from config import sf_username, sf_password, sf_security_token
from config import remote_db_endpoint, remote_db_port
from config import remote_db_name, remote_db_user, remote_db_pwd

In [3]:
from simple_salesforce import Salesforce
sf = Salesforce(username=sf_username, password=sf_password, security_token=sf_security_token)

In [4]:
engine = create_engine(f"mysql://{remote_db_user}:{remote_db_pwd}@{remote_db_endpoint}:{remote_db_port}/{remote_db_name}")
conn = engine.connect()

In [5]:
#Staff
staff_data_df = pd.read_sql("SELECT * FROM staff", conn)
staff_data_df.head(30)

,ID_Staff,EmployeeID,LastName,FirstName,MiddleName,BirthDate
0,1,000184220,Wimberly,Sam,Nico,1995-07-05
1,2,000130109,Sanford,Gemini,Blair,1992-04-22
2,3,000160655,Williams,Dartanion,De Angelo,1993-05-21
3,4,000159108,Popelka,Sarah,Nisan,1994-06-07


In [6]:
staff_data_records = staff_data_df.to_dict('records')
staff_data_records

[{'ID_Staff': 1,
  'EmployeeID': '000184220',
  'LastName': 'Wimberly',
  'FirstName': 'Sam',
  'MiddleName': 'Nico',
  'BirthDate': datetime.date(1995, 7, 5)},
 {'ID_Staff': 2,
  'EmployeeID': '000130109',
  'LastName': 'Sanford',
  'FirstName': 'Gemini',
  'MiddleName': 'Blair',
  'BirthDate': datetime.date(1992, 4, 22)},
 {'ID_Staff': 3,
  'EmployeeID': '000160655',
  'LastName': 'Williams',
  'FirstName': 'Dartanion',
  'MiddleName': 'De Angelo',
  'BirthDate': datetime.date(1993, 5, 21)},
 {'ID_Staff': 4,
  'EmployeeID': '000159108',
  'LastName': 'Popelka',
  'FirstName': 'Sarah',
  'MiddleName': 'Nisan',
  'BirthDate': datetime.date(1994, 6, 7)}]

In [7]:
for rec in staff_data_records:

    record = {
        'ID_Staff__c': rec['ID_Staff'],
        'EmployeeID__c': rec['EmployeeID'],
        'LastName__c': rec['LastName'],
        'FirstName__c': rec['FirstName'],
        'MiddleName__c': rec['MiddleName'],
        'BirthDate__c': rec['BirthDate'].isoformat(),
    }
    
    try:
        sf.Staff__c.create(record)
    except Exception as e:
        print(e)

In [8]:
staff_lookup_list = []

data = sf.query_all_iter("SELECT ID_Staff__c, Name FROM Staff__c")
for row in data:
    rec = {
        'ID_Staff': row['ID_Staff__c'],
        'StaffSF': row['Name']
    }
    staff_lookup_list.append(rec)

In [9]:
staff_lookup_df = pd.DataFrame(staff_lookup_list)
staff_lookup_df

,ID_Staff,StaffSF
0,4.0,a043h00000MVXKQ
1,2.0,a043h00000MVXKG
2,1.0,a043h00000MVXKB
3,3.0,a043h00000MVXKL


In [10]:
class_lookup_list = []

data = sf.query_all_iter("SELECT ID_Class__c, Name FROM Class__c")
for row in data:
    rec = {
        'ID_Class': row['ID_Class__c'],
        'ClassSF': row['Name']
    }
    class_lookup_list.append(rec)

In [11]:
class_lookup_df = pd.DataFrame(class_lookup_list)
class_lookup_df

""


In [12]:
#Staffassignment
query = '''
    SELECT
        sa.*
        
    FROM
        staffassignment sa
        
'''
staffassignment_data_df = pd.read_sql(query, conn)
staffassignment_data_df.head()

,ID_StaffAssignment,ID_Staff,ID_Class,Role,StartDate,EndDate
0,1,1,1,Teacher Assistant,2020-03-16,None
1,2,4,1,Teacher Assistant,2020-03-16,None
2,3,3,1,Instructor,2020-03-16,None


In [13]:
staffassignment_data_df = pd.merge(staffassignment_data_df, staff_lookup_df, how='left')
staffassignment_data_df.head()

,ID_StaffAssignment,ID_Staff,ID_Class,Role,StartDate,EndDate,StaffSF
0,1,1,1,Teacher Assistant,2020-03-16,None,a043h00000MVXKB
1,2,4,1,Teacher Assistant,2020-03-16,None,a043h00000MVXKQ
2,3,3,1,Instructor,2020-03-16,None,a043h00000MVXKL


In [14]:
staffassignment_data_df = pd.merge(staffassignment_data_df, class_lookup_df, how='left')
staffassignment_data_df.head()

MergeError: No common columns to perform merge on. Merge options: left_on=None, right_on=None, left_index=False, right_index=False

In [15]:
staffassignment_data_records = staffassignment_data_df.to_dict(orient='records')
staffassignment_data_records

[{'ID_StaffAssignment': 1,
  'ID_Staff': 1,
  'ID_Class': 1,
  'Role': 'Teacher Assistant',
  'StartDate': datetime.date(2020, 3, 16),
  'EndDate': None,
  'StaffSF': 'a043h00000MVXKB'},
 {'ID_StaffAssignment': 2,
  'ID_Staff': 4,
  'ID_Class': 1,
  'Role': 'Teacher Assistant',
  'StartDate': datetime.date(2020, 3, 16),
  'EndDate': None,
  'StaffSF': 'a043h00000MVXKQ'},
 {'ID_StaffAssignment': 3,
  'ID_Staff': 3,
  'ID_Class': 1,
  'Role': 'Instructor',
  'StartDate': datetime.date(2020, 3, 16),
  'EndDate': None,
  'StaffSF': 'a043h00000MVXKL'}]

In [16]:
for rec in staffassignment_data_records:
 
    record = {
        'ID_StaffAssignment__c': rec['ID_StaffAssignment'],
        'ID_Staff__c': rec['ID_Staff'],
        'ID_Class__c': rec['ID_Class'],
        'Role__c': rec['Role'],
        'StartDate__c': rec['StartDate'].isoformat(),
        'EndDate__c': rec['EndDate'],
        'Staff_Name__c': rec['StaffSF'],
        'Class_Name__c': rec['ClassSF']
    }
    
    try:
        sf.Staff_Assignment__c.create(record)
    except Exception as e:
        print(e)

KeyError: 'ClassSF'